# Functions

In [ ]:
import sys, pickle, copy
import scanpy as sc
from scipy.sparse import spmatrix, issparse, csr_matrix
from anndata import AnnData
from typing import Optional, Union
from shapely.geometry import Point, MultiPoint
import numpy as np
import pandas as pd
from tqdm import tqdm
from pathlib import Path
from scipy import linalg as LA
from math import cos, sin

# Simulation

## Analysis

In [ ]:
import pandas as pd
df = pd.read_csv('./results/Simulation/HMEC/SimulationResults.csv', header=0)
df['RE']=df['RMSD']**2*100/113.9  # RE = RMSD² × n_genes / ||C_benchmark||²
df['Resolution'] = (10000/df['Resolution']).astype(int)
df['CellNumber/Resolution'] = df['CellNumber'].astype(str) +'/'+ df['Resolution'].astype(str)
df

In [ ]:
print(df.RE.max())
print(df.RE.min())

In [ ]:
df[(df['CaptureRate'] == 0.5) & (df['DropRate'] == 0.3)].RMSD.median() - df[(df['CaptureRate'] == 0.1) & (df['DropRate'] == 0.6)].RMSD.mean()

In [ ]:
quantile_25_noise_5 = df[df['CaptureRate'] == 0.1].RE.quantile(0.25)
quantile_75_noise_5 = df[df['CaptureRate'] == 0.1].RE.quantile(0.75)
median_noise_5 = df[df['CaptureRate'] == 0.1].RE.median()

quantile_25_noise_0 = df[df['CaptureRate'] == 0.5].RE.quantile(0.25)
quantile_75_noise_0 = df[df['CaptureRate'] == 0.5].RE.quantile(0.75)
median_noise_0 = df[df['CaptureRate'] == 0.5].RE.median()

diff_median = median_noise_5 - median_noise_0

round(quantile_25_noise_5, 4), round(quantile_75_noise_5, 4), round(median_noise_5, 4), round(quantile_25_noise_0, 4), round(quantile_75_noise_0, 4), round(median_noise_0, 4), round(diff_median, 4)

In [ ]:
#quantile_25_noise_5 = df[df['DropRate'] == 0.3].RE.quantile(0.25)
#quantile_75_noise_5 = df[df['DropRate'] == 0.3].RE.quantile(0.75)
#median_noise_5 = df[df['DropRate'] == 0.3].RE.median()
#
#quantile_25_noise_0 = df[df['DropRate'] == 0.6].RE.quantile(0.25)
#quantile_75_noise_0 = df[df['DropRate'] == 0.6].RE.quantile(0.75)
#median_noise_0 = df[df['DropRate'] == 0.6].RE.median()
#
#diff_median = median_noise_5 - median_noise_0
#
#round(quantile_25_noise_5, 4), round(quantile_75_noise_5, 4), round(median_noise_5, 4), round(quantile_25_noise_0, 4), round(quantile_75_noise_0, 4), round(median_noise_0, 4), round(diff_median, 4)

In [ ]:
quantile_25_noise_5 = df[df['DropRate'] == 0.3].RE.quantile(0.25)
quantile_75_noise_5 = df[df['DropRate'] == 0.3].RE.quantile(0.75)
median_noise_5 = df[df['DropRate'] == 0.3].RE.median()

quantile_25_noise_0 = df[df['DropRate'] == 0.6].RE.quantile(0.25)
quantile_75_noise_0 = df[df['DropRate'] == 0.6].RE.quantile(0.75)
median_noise_0 = df[df['DropRate'] == 0.6].RE.median()

diff_median = median_noise_5 - median_noise_0

round(quantile_25_noise_5, 4), round(quantile_75_noise_5, 4), round(median_noise_5, 4), round(quantile_25_noise_0, 4), round(quantile_75_noise_0, 4), round(median_noise_0, 4), round(diff_median, 4)

In [ ]:
quantile_25_noise_5 = df[df['Noise'] == 5].RE.quantile(0.25)
quantile_75_noise_5 = df[df['Noise'] == 5].RE.quantile(0.75)
median_noise_5 = df[df['Noise'] == 5].RE.median()

quantile_25_noise_0 = df[df['Noise'] == 0].RE.quantile(0.25)
quantile_75_noise_0 = df[df['Noise'] == 0].RE.quantile(0.75)
median_noise_0 = df[df['Noise'] == 0].RE.median()

diff_median = median_noise_5 - median_noise_0

round(quantile_25_noise_5, 4), round(quantile_75_noise_5, 4), round(median_noise_5, 4), round(quantile_25_noise_0, 4), round(quantile_75_noise_0, 4), round(median_noise_0, 4), round(diff_median, 4)

In [ ]:
quantile_25_noise_5 = df[df['Resolution'] == 1000].RE.quantile(0.25)
quantile_75_noise_5 = df[df['Resolution'] == 1000].RE.quantile(0.75)
median_noise_5 = df[df['Resolution'] == 1000].RE.median()

quantile_25_noise_0 = df[df['Resolution'] == 333].RE.quantile(0.25)
quantile_75_noise_0 = df[df['Resolution'] == 333].RE.quantile(0.75)
median_noise_0 = df[df['Resolution'] == 333].RE.median()

diff_median = median_noise_5 - median_noise_0

round(quantile_25_noise_5, 4), round(quantile_75_noise_5, 4), round(median_noise_5, 4), round(quantile_25_noise_0, 4), round(quantile_75_noise_0, 4), round(median_noise_0, 4), round(diff_median, 4)

In [ ]:
quantile_25_noise_5 = df[df['CellNumber'] == 1000].RE.quantile(0.25)
quantile_75_noise_5 = df[df['CellNumber'] == 1000].RE.quantile(0.75)
median_noise_5 = df[df['CellNumber'] == 1000].RE.median()

quantile_25_noise_0 = df[df['CellNumber'] == 100].RE.quantile(0.25)
quantile_75_noise_0 = df[df['CellNumber'] == 100].RE.quantile(0.75)
median_noise_0 = df[df['CellNumber'] == 100].RE.median()

diff_median = median_noise_5 - median_noise_0

round(quantile_25_noise_5, 4), round(quantile_75_noise_5, 4), round(median_noise_5, 4), round(quantile_25_noise_0, 4), round(quantile_75_noise_0, 4), round(median_noise_0, 4), round(diff_median, 4)

In [ ]:
df[df['AnchorGenePercent'] == 40].RE.median() - df[df['AnchorGenePercent'] == 10].RE.median()

In [ ]:
df[df['AnchorGenePercent'] == 40].RE.median()

#### general

In [ ]:
df

In [ ]:
df_nonoise_highreso = df[(df['Noise'] == 5) & (df['Resolution'] == 333)]
median_re_value = round(df_nonoise_highreso.RE.median(), 4)
median_spearman_value = round(df_nonoise_highreso.SPEARMAN.median(), 4)
median_re_value, median_spearman_value

In [ ]:
df_nonoise_highreso = df[(df['Noise'] == 5) & (df['Resolution'] == 333)]
median_re_value = round(df_nonoise_highreso.RE.median(), 4)
median_spearman_value = round(df_nonoise_highreso.SPEARMAN.median(), 4)
median_re_value, median_spearman_value

In [ ]:
df_highnoise = df[df['Noise'] != 5]
median_value = round(df_highnoise.SPEARMAN.median(), 4)
quantile_75 = round(df_highnoise.SPEARMAN.quantile(0.75), 4)
quantile_25 = round(df_highnoise.SPEARMAN.quantile(0.25), 4)
median_value, quantile_25, quantile_75

In [ ]:
median_value = round(df_highnoise.RE.median(), 4)
quantile_75 = round(df_highnoise.RE.quantile(0.75), 4)
quantile_25 = round(df_highnoise.RE.quantile(0.25), 4)
median_value, quantile_25, quantile_75

In [ ]:
median_value = round(df.SPEARMAN.median(), 4)
quantile_75 = round(df.SPEARMAN.quantile(0.75), 4)
quantile_25 = round(df.SPEARMAN.quantile(0.25), 4)
median_value, quantile_25, quantile_75

In [ ]:
median_value = round(df.RE.median(), 4)
quantile_75 = round(df.RE.quantile(0.75), 4)
quantile_25 = round(df.RE.quantile(0.25), 4)
median_value, quantile_25, quantile_75

### Fig2b. Panel Plot

In [ ]:
def get_cmap(cmap, ncolor):
    # Import matplotlib
    import matplotlib.pyplot as plt
    import matplotlib.colors as mcolors

    # Define the color map
    cmap = plt.get_cmap(cmap)

    # Get the RGB values of each color in the color map
    hex_values=[]
    for i in range(ncolor):
        hex_values.append(mcolors.rgb2hex(cmap(int(256/ncolor)*i)))

    return hex_values

get_cmap('Greens',4)

In [ ]:
df_sub

In [ ]:
import seaborn as sns
import numpy as np

import matplotlib.pyplot as plt

# getting unique values of Drop rate and Capture rate
# df = df.query('AnchorGenePercent==10')

droprates = df['DropRate'].unique()
capturerates = df['CaptureRate'].unique()

# setting the plot dimension
fig, axs = plt.subplots(len(capturerates), len(droprates), figsize=(20,10), dpi=600)

# iterating through each Subplot
for i, capturerate in enumerate(capturerates):
    for j, droprate in enumerate(droprates):
        
        # subsetting data for each subplot
        subset = df[(df['DropRate'] == droprate) & (df['CaptureRate'] == capturerate)]
        
        # creating a strip plot for the subplot
        sns.boxplot(x="Noise", y="RE", hue="CellNumber/Resolution", data=subset, ax=axs[i][j], linewidth=0.5, flierprops={"marker":'x', "markersize":2.5},
                    hue_order=['1000/333','500/333','100/333','1000/500','500/500','100/500','1000/1000','500/1000','100/1000'], 
                    palette={'1000/333': "#2070b4", '500/333': "#6aaed6", '100/333': "#c6dbef",
                            '1000/500': "#228a44", '500/500': "#73c476", '100/500': "#c7e9c0",
                            '1000/1000': "#ca181d", '500/1000': "#fb694a", '100/1000': "#fcbba1",})
        
        # setting title
        axs[i][j].set_title(f'Detection Efficiency: {capturerate*100}% | Gene Dropout Rate: {droprate*100}%', fontsize=11)
        axs[i][j].set_xticklabels(["None","Low","High"])
        axs[i][j].set_xlabel("Noise level", fontsize=12)
        axs[i][j].set_ylabel("Relative error", fontsize=12)
        

        # removing the legend
        axs[i][j].legend([],[], frameon=False)
        # set y-axis to log10 scale
        axs[i][j].set_yscale('log', base=10, subs=[0,1,2,3])
        axs[i][j].set_ylim(0.00001,3.2)
        if i==0 and j==3:
            axs[i][j].legend(loc='lower center', title="Cell Number / Resolution (nm)", ncol=3)   

# setting the layout
plt.tight_layout()
plt.savefig('figures/Fig2c.pdf', bbox_inches='tight', dpi=400)
#plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# getting unique values of Drop rate and Capture rate
df_sub = df.query('AnchorGenePercent==10')

droprates = df_sub['DropRate'].unique()
capturerates = df_sub['CaptureRate'].unique()

# setting the plot dimension
fig, axs = plt.subplots(len(capturerates), len(droprates), figsize=(20,10), dpi=600)

# iterating through each Subplot
for i, capturerate in enumerate(capturerates):
    for j, droprate in enumerate(droprates):
        
        # subsetting data for each subplot
        subset = df_sub[(df_sub['DropRate'] == droprate) & (df_sub['CaptureRate'] == capturerate)]
        
        # creating a strip plot for the subplot
        sns.boxplot(x="Noise", y="RE", hue="CellNumber/Resolution", data=subset, ax=axs[i][j], linewidth=0.5, flierprops={"marker":'x', "markersize":1},
                    hue_order=['1000/333','500/333','100/333','1000/500','500/500','100/500','1000/1000','500/1000','100/1000'], 
                    palette={'1000/333': "#2070b4", '500/333': "#6aaed6", '100/333': "#c6dbef",
                            '1000/500': "#228a44", '500/500': "#73c476", '100/500': "#c7e9c0",
                            '1000/1000': "#ca181d", '500/1000': "#fb694a", '100/1000': "#fcbba1",})
        
        # setting title
        axs[i][j].set_title(f'Detection Efficiency: {capturerate*100}% | Gene Dropout Rate: {droprate*100}%', fontsize=11)
        axs[i][j].set_xticklabels(["None","Low","High"])
        axs[i][j].set_xlabel("Noise Level")
        axs[i][j].set_ylabel("Relative error")
        axs[i][j].set_ylim(0,0.65)
        axs[i][j].legend(loc='upper left', title="Cell Number / Resolution (nm)", ncol=3, fontsize=9.2)
        
# setting the layout
plt.tight_layout()
plt.show()

### Bar Plot for Mirror

In [ ]:
df

In [ ]:
df.columns

In [ ]:
# group by 'group1' and 'group2', calculate sum of 'mirror', and count of rows (not including NaNs in 'mirror')
grouped = df.groupby(['Noise','CaptureRate','DropRate','Resolution','CellNumber']).agg(
    mirror_sum=('MIRROR', 'sum'),
    count=('MIRROR', 'count')
)

# calculate 'mirror percent'
grouped['mirror_percent'] = grouped['mirror_sum'] / grouped['count']
print(pd.DataFrame([grouped['mirror_sum'],grouped['count'],grouped['mirror_percent']]).T.mirror_percent.values)

In [ ]:
df

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

args=['CellNumber', 'CaptureRate', 'DropRate', 'Resolution', 'AnchorGenePercent', 'Noise']
xlabels=['Cell Number', 'Detection Efficiency', 'Gene Dropout Rate', 'Resolution', 'Anchor Gene Rate', 'Noise Level']
# setting the plot dimension
fig, axs = plt.subplots(1,len(args), figsize=(12,2), dpi=300)

# iterating through each Subplot
for i, arg in enumerate(args):
        # group by arg, calculate sum of 'mirror', and count of rows (not including NaNs in 'mirror')
        grouped = df.groupby([arg]).agg(
                mirror_sum=('MIRROR', 'sum'),
                count=('MIRROR', 'count')
        )

        # calculate 'mirror percent'
        grouped['mirror_percent'] = grouped['mirror_sum'] / grouped['count']
        df_mirror = pd.DataFrame([grouped['mirror_sum'],grouped['count'],grouped['mirror_percent']])
        
        # creating a strip plot for the subplot
        sns.barplot(x=df_mirror.T.index.values, y=df_mirror.T.mirror_percent.values, ax=axs[i], palette='Blues')
        
        # setting title
        #axs[i].set_title(f'Capture Rate: {capturerate*100}% | Drop Rate: {droprate*100}%')
        #axs[i].set_xticklabels(["None","Low","High"])
        axs[i].set_xlabel(xlabels[i])
        axs[i].set_ylabel("Mirror Rate")
        axs[i].set_ylim(0,1)
        #axs[i].legend(loc='upper left', title="Cell Number / Resolution", ncol=3)
        
# setting the layout
plt.tight_layout()
plt.savefig('figures/Simulation_MirrorRate.png', dpi=300)
plt.show()

### Fig2c. Comparison

In [ ]:
import pandas as pd
df = pd.read_csv('./results/Simulation/HMEC/SimulationResults.csv', header=0)
df['RE']=df['RMSD']**2*100/113.9  # RE = RMSD² × n_genes / ||C_benchmark||²
df['Resolution'] = (10000/df['Resolution']).astype(int)
df['CellNumber/Resolution'] = df['CellNumber'].astype(str) +'/'+ df['Resolution'].astype(str)
df['Method'] = 'cytocraft'
df

In [ ]:
df_compare = pd.read_csv('./results/Simulation/HMEC/compare_baseline/SimulationCompareResults.csv', header=None)
cols = df.columns.tolist()
cols[-3] = 'Method'
# assign matching number of column names from df to df_compare
df_compare.columns = cols[: df_compare.shape[1]] if len(cols) != df_compare.shape[1] else cols

In [ ]:
df_compare['RE']=df_compare['RMSD']**2*100/113.9  # RE = RMSD² × n_genes / ||C_benchmark||²
df_compare['Resolution'] = (10000/df_compare['Resolution']).astype(int)
df_compare['CellNumber/Resolution'] = df_compare['CellNumber'].astype(str) +'/'+ df_compare['Resolution'].astype(str)
df_compare

In [ ]:
# merge df and df_compare by concatenation (keep all columns)
df_merged = pd.concat([df, df_compare], ignore_index=True)
df_merged

In [ ]:
import os
import matplotlib.pyplot as plt
import seaborn as sns

# combined plot: x = evaluation metric, hue = Method
metrics = ['RE', 'SPEARMAN']

# melt dataframe into long form
melted = df_merged[['Method'] + metrics].melt(id_vars='Method',
                                             value_vars=metrics,
                                             var_name='Evaluation',
                                             value_name='Value')
# drop missing and remove non-positive RE for clarity
melted = melted.dropna(subset=['Value'])
melted = melted[~((melted['Evaluation'] == 'RE') & (melted['Value'] <= 0))]

# define method order (optional: use a specific order like order_re/order_spear)
method_order = list(df_merged['Method'].unique())

plt.figure(figsize=(4, 3), dpi=300)
palette = {'cytocraft': '#E64B34', 'average': '#4EB9D4', 'isomap_k10': '#37A089'}
sns.boxplot(x='Evaluation', y='Value', hue='Method', data=melted,
            hue_order=method_order, showfliers=False, palette=palette,
            width=0.6, linewidth=0.8)

# remove duplicate legend (stripplot adds a second legend)
handles, labels = plt.gca().get_legend_handles_labels()
# keep only the first block of handles (one per method)
n_methods = len(method_order)
plt.legend(handles[:n_methods], labels[:n_methods], title='Method', bbox_to_anchor=(1.05, 1), loc='upper left')

# set x tick labels and y limits
plt.gca().set_xticklabels(['Relative error', 'Spearman'])
plt.xlabel('')
plt.ylabel('')
plt.ylim(-0.1, 1.1)
plt.tight_layout()
plt.savefig('figures/evaluations_by_method.pdf', bbox_inches='tight')
plt.show()


In [ ]:
# draw one combined RE / SPEARMAN plot for each noise level
noise_levels = sorted(df_merged['Noise'].unique())
n = len(noise_levels)
fig_width = max(3.1 * n, 4)
fig, axs = plt.subplots(1, n, figsize=(fig_width, 3), dpi=300, sharey=True)

if n == 1:
    axs = [axs]

for ax, noise in zip(axs, noise_levels):
    subset = df_merged[df_merged['Noise'] == noise]
    melted_noise = subset[['Method'] + metrics].melt(id_vars='Method',
                                                     value_vars=metrics,
                                                     var_name='Evaluation',
                                                     value_name='Value')
    melted_noise = melted_noise.dropna(subset=['Value'])
    melted_noise = melted_noise[~((melted_noise['Evaluation'] == 'RE') & (melted_noise['Value'] <= 0))]

    sns.boxplot(x='Evaluation', y='Value', hue='Method', data=melted_noise,
                hue_order=method_order, showfliers=False, palette=palette,
                width=0.6, linewidth=0.8, ax=ax)

    ax.set_title(f'Noise = {noise}')
    ax.set_xlabel('')
    ax.set_ylabel('')  # only show ylabel on the first subplot
    ax.set_ylim(-0.1, 1.1)
    ax.set_xticklabels(['Relative error', 'Spearman'])

    # only show legend on the first subplot (keep it outside)
    if ax is axs[0]:
        handles, labels = ax.get_legend_handles_labels()
        ax.legend(handles[:len(method_order)], labels[:len(method_order)],
                  title='Method', bbox_to_anchor=(1.02, 1), loc='upper left')
    else:
        ax.get_legend().remove()

plt.tight_layout()
plt.savefig('figures/evaluations_by_method_by_noise.pdf', bbox_inches='tight')
plt.show()


In [ ]:
import os
import matplotlib.pyplot as plt
import seaborn as sns

# Create a 2x2 subplot grid
fig, axs = plt.subplots(2, 2, figsize=(5.2, 6), dpi=300)
axs = axs.flatten()  # Flatten the 2D array of axes for easier iteration

# --- Plot 1: Overall Comparison (Top-Left Panel) ---
metrics = ['RE', 'SPEARMAN']
melted = df_merged[['Method'] + metrics].melt(id_vars='Method',
                                             value_vars=metrics,
                                             var_name='Evaluation',
                                             value_name='Value')
melted = melted.dropna(subset=['Value'])
melted = melted[~((melted['Evaluation'] == 'RE') & (melted['Value'] <= 0))]

method_order = list(df_merged['Method'].unique())
palette = {'cytocraft': '#E64B34', 'average': '#4EB9D4', 'isomap_k10': '#37A089'}

sns.boxplot(x='Evaluation', y='Value', hue='Method', data=melted,
            hue_order=method_order, showfliers=False, palette=palette,
            width=0.6, linewidth=0.8, ax=axs[0])

axs[0].set_title('Overall Comparison')
axs[0].set_xticklabels(['Relative error', 'Spearman'])
axs[0].set_xlabel('')
axs[0].set_ylabel('')
axs[0].set_ylim(-0.1, 1.1)
axs[0].get_legend().remove() # Remove individual legend

# --- Plots 2, 3, 4: Comparison by Noise Level ---
noise_levels = sorted(df_merged['Noise'].unique())

# Use the next 3 axes for the noise-level plots
for i, noise in enumerate(noise_levels[:3]):
    ax = axs[i + 1]
    subset = df_merged[df_merged['Noise'] == noise]
    melted_noise = subset[['Method'] + metrics].melt(id_vars='Method',
                                                     value_vars=metrics,
                                                     var_name='Evaluation',
                                                     value_name='Value')
    melted_noise = melted_noise.dropna(subset=['Value'])
    melted_noise = melted_noise[~((melted_noise['Evaluation'] == 'RE') & (melted_noise['Value'] <= 0))]

    sns.boxplot(x='Evaluation', y='Value', hue='Method', data=melted_noise,
                hue_order=method_order, showfliers=False, palette=palette,
                width=0.6, linewidth=0.8, ax=ax)

    ax.set_title(f'Noise = {noise}')
    ax.set_xlabel('')
    ax.set_ylabel('')
    ax.set_ylim(-0.1, 1.1)
    ax.set_xticklabels(['Relative error', 'Spearman'])
    if ax.get_legend() is not None:
        ax.get_legend().remove()

# --- Final Figure Adjustments ---
# Create a single legend for the entire figure
# handles, labels = axs[0].get_legend_handles_labels()
# fig.legend(handles, labels, title='Method', bbox_to_anchor=(0.9, 0.5), loc='center left')

plt.tight_layout(rect=[0, 0, 0.9, 1]) # Adjust layout to make space for the legend
plt.savefig('figures/Fig2c.combined_evaluations.pdf', bbox_inches='tight')
plt.show()


In [ ]:
import pandas as pd

# Verify statistical statement about performance differences among methods across noise levels

# Ensure required dataframe exists
assert 'df_merged' in globals(), "df_merged not found. Run previous comparison cells first."

# Identify method names
methods = df_merged['Method'].unique().tolist()
assert 'cytocraft' in methods, "cytocraft method missing."
avg_method = 'average' if 'average' in methods else None
iso_methods = [m for m in methods if 'isomap' in m.lower()]
iso_method = iso_methods[0] if iso_methods else None
assert avg_method is not None, "Average baseline missing."
assert iso_method is not None, "Isomap baseline missing."

# Count simulations
total_runs = len(df_merged)
runs_per_noise = df_merged.groupby('Noise').size().to_dict()

# Median stats by noise and method
median_table = (
    df_merged.groupby(['Noise', 'Method'])[['RE', 'SPEARMAN']]
    .median()
    .unstack('Method')
)

# Helper to fetch values
def get_val(metric, method, noise):
    return median_table.loc[noise, (metric, method)]

noise_levels = median_table.index.tolist()

# Compute fold reductions (RE) and absolute improvements (SPEARMAN)
rows = []
for n in noise_levels:
    re_c = get_val('RE', 'cytocraft', n)
    re_a = get_val('RE', avg_method, n)
    re_i = get_val('RE', iso_method, n)
    sp_c = get_val('SPEARMAN', 'cytocraft', n)
    sp_a = get_val('SPEARMAN', avg_method, n)
    sp_i = get_val('SPEARMAN', iso_method, n)
    rows.append({
        'Noise': n,
        'Median_RE_cytocraft': re_c,
        f'Median_RE_{avg_method}': re_a,
        f'Median_RE_{iso_method}': re_i,
        'Fold_RE_vs_Average': re_a / re_c if re_c > 0 else float('inf'),
        'Fold_RE_vs_Isomap': re_i / re_c if re_c > 0 else float('inf'),
        'Delta_Sp_vs_Average': sp_c - sp_a,
        'Delta_Sp_vs_Isomap': sp_c - sp_i
    })

verification_df = pd.DataFrame(rows)

# Ranges
fold_avg_min, fold_avg_max = verification_df['Fold_RE_vs_Average'].min(), verification_df['Fold_RE_vs_Average'].max()
fold_iso_min, fold_iso_max = verification_df['Fold_RE_vs_Isomap'].min(), verification_df['Fold_RE_vs_Isomap'].max()
delta_sp_avg_min, delta_sp_avg_max = verification_df['Delta_Sp_vs_Average'].min(), verification_df['Delta_Sp_vs_Average'].max()
delta_sp_iso_min, delta_sp_iso_max = verification_df['Delta_Sp_vs_Isomap'].min(), verification_df['Delta_Sp_vs_Isomap'].max()

# Claimed ranges
claim_fold_avg = (4.2, 11.7)
claim_fold_iso = (2.9, 7.5)
claim_sp_avg = (0.12, 0.31)
claim_sp_iso = (0.08, 0.24)

def within(observed_min, observed_max, claimed_min, claimed_max):
    return observed_min >= claimed_min - 1e-9 and observed_max <= claimed_max + 1e-9

result = {
    'Total simulations (rows)': total_runs,
    'Runs per noise': runs_per_noise,
    'Observed fold RE vs Average (min, max)': (round(fold_avg_min,3), round(fold_avg_max,3)),
    'Observed fold RE vs Isomap (min, max)': (round(fold_iso_min,3), round(fold_iso_max,3)),
    'Observed ΔSpearman vs Average (min, max)': (round(delta_sp_avg_min,3), round(delta_sp_avg_max,3)),
    'Observed ΔSpearman vs Isomap (min, max)': (round(delta_sp_iso_min,3), round(delta_sp_iso_max,3)),
    'Claim match RE fold vs Average': within(fold_avg_min, fold_avg_max, *claim_fold_avg),
    'Claim match RE fold vs Isomap': within(fold_iso_min, fold_iso_max, *claim_fold_iso),
    'Claim match ΔSpearman vs Average': within(delta_sp_avg_min, delta_sp_avg_max, *claim_sp_avg),
    'Claim match ΔSpearman vs Isomap': within(delta_sp_iso_min, delta_sp_iso_max, *claim_sp_iso),
}

print("Per-noise medians and derived stats:")
print(verification_df.round(4).to_string(index=False))
print("\nSummary verification:")
for k,v in result.items():
    print(f"{k}: {v}")

# If any mismatch, flag
if not all([result['Claim match RE fold vs Average'],
            result['Claim match RE fold vs Isomap'],
            result['Claim match ΔSpearman vs Average'],
            result['Claim match ΔSpearman vs Isomap']]):
    print("\nConclusion: Provided statement does NOT fully match observed ranges.")
else:
    print("\nConclusion: Provided statement is consistent with observed statistics.")

In [ ]:
import pandas as pd

# Verify statistical statement about performance differences among methods across noise levels

# Ensure required dataframe exists
assert 'df_merged' in globals(), "df_merged not found. Run previous comparison cells first."

# Identify method names
methods = df_merged['Method'].unique().tolist()
assert 'cytocraft' in methods, "cytocraft method missing."
avg_method = 'average' if 'average' in methods else None
iso_methods = [m for m in methods if 'isomap' in m.lower()]
iso_method = iso_methods[0] if iso_methods else None
assert avg_method is not None, "Average baseline missing."
assert iso_method is not None, "Isomap baseline missing."

# Count simulations
total_runs = len(df_merged)
runs_per_noise = df_merged.groupby('Noise').size().to_dict()

# Median stats by noise and method
median_table = (
    df_merged.groupby(['Noise', 'Method'])[['RE', 'SPEARMAN']]
    .median()
    .unstack('Method')
)

# Helper to fetch values
def get_val(metric, method, noise):
    return median_table.loc[noise, (metric, method)]

noise_levels = median_table.index.tolist()

# Compute fold reductions (RE) and absolute improvements (SPEARMAN)
rows = []
for n in noise_levels:
    re_c = get_val('RE', 'cytocraft', n)
    re_a = get_val('RE', avg_method, n)
    re_i = get_val('RE', iso_method, n)
    sp_c = get_val('SPEARMAN', 'cytocraft', n)
    sp_a = get_val('SPEARMAN', avg_method, n)
    sp_i = get_val('SPEARMAN', iso_method, n)
    rows.append({
        'Noise': n,
        'Median_RE_cytocraft': re_c,
        f'Median_RE_{avg_method}': re_a,
        f'Median_RE_{iso_method}': re_i,
        'Fold_RE_vs_Average': re_a / re_c if re_c > 0 else float('inf'),
        'Fold_RE_vs_Isomap': re_i / re_c if re_c > 0 else float('inf'),
        'Delta_Sp_vs_Average': sp_c - sp_a,
        'Delta_Sp_vs_Isomap': sp_c - sp_i
    })

verification_df = pd.DataFrame(rows)

# Ranges
fold_avg_min, fold_avg_max = verification_df['Fold_RE_vs_Average'].min(), verification_df['Fold_RE_vs_Average'].max()
fold_iso_min, fold_iso_max = verification_df['Fold_RE_vs_Isomap'].min(), verification_df['Fold_RE_vs_Isomap'].max()
delta_sp_avg_min, delta_sp_avg_max = verification_df['Delta_Sp_vs_Average'].min(), verification_df['Delta_Sp_vs_Average'].max()
delta_sp_iso_min, delta_sp_iso_max = verification_df['Delta_Sp_vs_Isomap'].min(), verification_df['Delta_Sp_vs_Isomap'].max()

# Claimed ranges
claim_fold_avg = (4.2, 11.7)
claim_fold_iso = (2.9, 7.5)
claim_sp_avg = (0.12, 0.31)
claim_sp_iso = (0.08, 0.24)

def within(observed_min, observed_max, claimed_min, claimed_max):
    return observed_min >= claimed_min - 1e-9 and observed_max <= claimed_max + 1e-9

result = {
    'Total simulations (rows)': total_runs,
    'Runs per noise': runs_per_noise,
    'Observed fold RE vs Average (min, max)': (round(fold_avg_min,3), round(fold_avg_max,3)),
    'Observed fold RE vs Isomap (min, max)': (round(fold_iso_min,3), round(fold_iso_max,3)),
    'Observed ΔSpearman vs Average (min, max)': (round(delta_sp_avg_min,3), round(delta_sp_avg_max,3)),
    'Observed ΔSpearman vs Isomap (min, max)': (round(delta_sp_iso_min,3), round(delta_sp_iso_max,3)),
    'Claim match RE fold vs Average': within(fold_avg_min, fold_avg_max, *claim_fold_avg),
    'Claim match RE fold vs Isomap': within(fold_iso_min, fold_iso_max, *claim_fold_iso),
    'Claim match ΔSpearman vs Average': within(delta_sp_avg_min, delta_sp_avg_max, *claim_sp_avg),
    'Claim match ΔSpearman vs Isomap': within(delta_sp_iso_min, delta_sp_iso_max, *claim_sp_iso),
}

print("Per-noise medians and derived stats:")
print(verification_df.round(4).to_string(index=False))
print("\nSummary verification:")
for k,v in result.items():
    print(f"{k}: {v}")

# If any mismatch, flag
if not all([result['Claim match RE fold vs Average'],
            result['Claim match RE fold vs Isomap'],
            result['Claim match ΔSpearman vs Average'],
            result['Claim match ΔSpearman vs Isomap']]):
    print("\nConclusion: Provided statement does NOT fully match observed ranges.")
else:
    print("\nConclusion: Provided statement is consistent with observed statistics.")

In [ ]:
# Revised, data-driven performance description based on observed statistics
noise_label = {0: "none", 2: "low", 5: "high"}

# Extract per-noise rows
rows_map = {r.Noise: r for _, r in verification_df.iterrows()}

n_total = total_runs
re_avg_range = (round(fold_avg_min, 3), round(fold_avg_max, 3))
re_iso_range = (round(fold_iso_min, 3), round(fold_iso_max, 3))
sp_avg_range = (round(delta_sp_avg_min, 3), round(delta_sp_avg_max, 3))
sp_iso_range = (round(delta_sp_iso_min, 3), round(delta_sp_iso_max, 3))

def fmt_row(noise):
    r = rows_map[noise]
    return (f"Noise {noise_label[noise]}: RE fold vs Average={r.Fold_RE_vs_Average:.3f}, "
            f"RE fold vs Isomap={r.Fold_RE_vs_Isomap:.3f}, "
            f"ΔSpearman vs Average={r.Delta_Sp_vs_Average:+.3f}, "
            f"ΔSpearman vs Isomap={r.Delta_Sp_vs_Isomap:+.3f}")

per_noise_lines = "\n".join(fmt_row(n) for n in noise_levels)

revised_description = f"""
Across {n_total} simulations (noise levels: none, low, high):
• Median relative error (RE) fold change vs Average baseline spans {re_avg_range[0]}–{re_avg_range[1]}:
  – Strong improvements at none ({rows_map[0].Fold_RE_vs_Average:.2f}x) and low ({rows_map[2].Fold_RE_vs_Average:.2f}x) noise
  – Worse than Average at high noise (fold {rows_map[5].Fold_RE_vs_Average:.2f} < 1)
• Median RE fold improvement vs Isomap spans {re_iso_range[0]}–{re_iso_range[1]} (improvement at all noise levels)
• Median Spearman change vs Average ranges {sp_avg_range[0]} to {sp_avg_range[1]} (gains at none/low noise, loss at high noise)
• Median Spearman gain vs Isomap is consistently positive ({sp_iso_range[0]}–{sp_iso_range[1]})
Per-noise detail:
{per_noise_lines}
"""

print(revised_description.strip())